In [1]:
#####기본 분데이터 크롤링
import time
import datetime
import pandas as pd
import requests

url = 'https://www.binance.com/fapi/v1/continuousKlines?interval=1m&contractType=PERPETUAL&pair=XRPUSDT'
webpage = requests.get(url) #(1) 인터넷을 통해 데이터 받아오기
webpage.content             #(2) 데이터 출력

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))

In [14]:
#####120일 전 데이터 가져오기
base_url = "https://www.binance.com/fapi/v1/klines?symbol=XRPUSDT&interval=1m&limit=1000&startTime={}"
gettimestamp = int(time.time() - 60*60*24 * 120)*1000 #(2) 120일 시간지정
base_url = base_url.format(gettimestamp)               #(3) startTime 항목에 시간 설정
webpage = requests.get(base_url)                       
webpage.content

b'[[1614390720000,"0.4407","0.4413","0.4405","0.4406","340573.7",1614390779999,"150147.09833",201,"160437.5","70739.58719","0"],[1614390780000,"0.4406","0.4411","0.4404","0.4408","284910.7",1614390839999,"125555.19340",188,"137310.2","60521.81319","0"],[1614390840000,"0.4408","0.4412","0.4405","0.4409","218617.0",1614390899999,"96387.28947",180,"125986.8","55546.66737","0"],[1614390900000,"0.4408","0.4412","0.4406","0.4406","212898.8",1614390959999,"93854.49026",132,"80481.6","35482.65595","0"],[1614390960000,"0.4407","0.4410","0.4406","0.4408","111763.5",1614391019999,"49259.14068",102,"69024.5","30423.06033","0"],[1614391020000,"0.4410","0.4411","0.4407","0.4409","89625.2",1614391079999,"39519.11007",100,"48647.6","21453.24233","0"],[1614391080000,"0.4409","0.4411","0.4407","0.4407","153074.3",1614391139999,"67489.51940",89,"89364.1","39404.93743","0"],[1614391140000,"0.4408","0.4413","0.4405","0.4413","318392.0",1614391199999,"140361.45233",172,"123284.8","54355.74019","0"],[1614391

In [15]:
#####데이터프레임으로 전환
df_candle_temp = pd.read_json(webpage.content)
df_candle_temp

,0,1,2,3,4,5,6,7,8,9,10,11
0,1614390720000,0.4407,0.4413,0.4405,0.4406,340573.7,1614390779999,150147.09833,201,160437.5,70739.58719,0
1,1614390780000,0.4406,0.4411,0.4404,0.4408,284910.7,1614390839999,125555.19340,188,137310.2,60521.81319,0
2,1614390840000,0.4408,0.4412,0.4405,0.4409,218617.0,1614390899999,96387.28947,180,125986.8,55546.66737,0
3,1614390900000,0.4408,0.4412,0.4406,0.4406,212898.8,1614390959999,93854.49026,132,80481.6,35482.65595,0
4,1614390960000,0.4407,0.4410,0.4406,0.4408,111763.5,1614391019999,49259.14068,102,69024.5,30423.06033,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1614450420000,0.4454,0.4455,0.4451,0.4455,201504.1,1614450479999,89741.33014,85,132175.7,58868.84038,0
996,1614450480000,0.4455,0.4456,0.4454,0.4455,141224.8,1614450539999,62915.29990,96,70415.3,31371.08792,0
997,1614450540000,0.4455,0.4460,0.4455,0.4459,165253.8,1614450599999,73650.84480,123,141236.1,62945.59093,0
998,1614450600000,0.4459,0.4473,0.4459,0.4471,1102117.8,1614450659999,492134.93414,497,934677.7,417380.26592,0


In [34]:
#####timestamp 데이터 확인하기

def get_date(mili_time):
    KST = datetime.timezone(datetime.timedelta(hours=9))
    dt = datetime.datetime.fromtimestamp(mili_time / 1000.0, tz=KST)
    timeline = str(dt.strftime('%D %H:%M:%S'))  #(1) 출력형식 지정
    return timeline
get_date(1614450660000)

'02/28/21 03:31:00'

In [31]:
#####120일 전체 데이터 받기

base_url = "https://www.binance.com/fapi/v1/klines?symbol=XRPUSDT"+ \
                "&interval=1m&limit=1000&startTime={}"
gettimestamp = int(time.time() - 60*60*24 * 120)*1000

for i in range(int(120*1.4)): #(1) 데이터 건수 설정
    
    #(2)타임스템프 설정
    url = base_url.format(int(gettimestamp))       
    
    webpage = requests.get(url)
    
    #(3)JSON 형식 데이터 읽어서 임시 데이터프레임에 저장
    df_candle_temp = pd.read_json(webpage.content) 

    #(4) 새로받은 데이터를 기존 데이터프레임과 병합
    df_candle = pd.concat([df_candle,df_candle_temp],axis=0) 
    
    #(5)마지막 타임스템프 추출
    gettimestamp = df_candle_temp[0][-1:].values[0] 
    
#(6)컬럼명 수정
rename_columns = {0: 't', 1: 'o', 2: 'h', 3: 'l', 4: 'c', 5: 'v'}
df_candle = df_candle[[0, 1, 2, 3, 4, 5]].rename(columns=rename_columns)

#(7)널 데이터 삭제
df_candle = df_candle.dropna(axis=0)

#(8)파일로 저장
df_candle.to_csv("./data/XRPUSDT.csv", index=False)

In [44]:
#####최종 데이터 확인

gettimestamp = df_candle['t'][-1:].values[0]
get_date(gettimestamp)

'06/24/21 00:19:00'